In [1]:
import sounddevice as sd
import numpy as np
import librosa
import librosa.display
import soundfile as sf
import tkinter as tk
from tkinter import messagebox, ttk
from tensorflow.keras.models import load_model
import pickle
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.metrics import confusion_matrix
import seaborn as sns
from pydub import AudioSegment
import requests
import random
import datetime

# Load pre-trained model and label encoder
model = load_model('MSTCNN_model.h5')
with open('label_encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

# Define parameters
SAMPLE_RATE = 22050
DURATION = 30
NUM_MFCC = 13
N_FFT = 2048
HOP_LENGTH = 512
EXPECTED_MFCC_LEN = 1320

# Variable to store last recorded audio and classification history
last_audio = None
classification_history = []

# Tkinter root setup
root = tk.Tk()
root.title("Music Genre Classifier")
root.geometry("900x600")
root.config(bg="#1e1e2e")

# Notebook (Tabbed layout)
notebook = ttk.Notebook(root)
notebook.pack(expand=True, fill='both', pady=10)

# Tabs
tab_main = ttk.Frame(notebook)
tab_visualization = ttk.Frame(notebook)
tab_results = ttk.Frame(notebook)
notebook.add(tab_main, text="Record & Classify")
notebook.add(tab_visualization, text="Visualizations")
notebook.add(tab_results, text="Results")

# Styles
style = ttk.Style(root)
style.theme_use("clam")
style.configure("TButton", padding=6, relief="flat", background="#2e2e3e", foreground="white", font=("Helvetica", 10))
style.configure("TLabel", background="#1e1e2e", foreground="white", font=("Helvetica", 12))
style.configure("TNotebook", background="#2e2e3e", foreground="white")
style.configure("TNotebook.Tab", font=("Helvetica", 12), padding=[10, 5])

# Record audio
def record_audio(duration=DURATION, sr=SAMPLE_RATE):
    global last_audio
    status_label.config(text="Recording...")
    audio = sd.rec(int(duration * sr), samplerate=sr, channels=1)
    sd.wait()
    last_audio = np.squeeze(audio)
    status_label.config(text="Recording finished.")
    return last_audio

# Preprocess audio to MFCC
def preprocess_audio(audio, sr=SAMPLE_RATE, num_mfcc=NUM_MFCC, n_fft=N_FFT, hop_length=HOP_LENGTH, expected_len=EXPECTED_MFCC_LEN):
    mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
    if mfcc.shape[1] < expected_len:
        mfcc = np.pad(mfcc, ((0, 0), (0, expected_len - mfcc.shape[1])), mode='constant')
    else:
        mfcc = mfcc[:, :expected_len]
    return np.expand_dims(mfcc.T, axis=0)

# Classify genre
def classify_genre(audio_mfcc):
    prediction = model.predict(audio_mfcc)
    genre_index = np.argmax(prediction, axis=1)
    genre_label = encoder.inverse_transform(genre_index)
    confidence = prediction[0][genre_index[0]] * 100
    return genre_label[0], confidence

# Classify button function
def classify_audio():
    audio = record_audio()
    audio_mfcc = preprocess_audio(audio)
    genre, confidence = classify_genre(audio_mfcc)
    
    classification_history.append((datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), genre, confidence))
    update_history_table()
    
    result_label.config(text=f"Predicted Genre: {genre} \nConfidence: {confidence:.2f}%")
    status_label.config(text="Classification Complete")

    # Fetch song suggestions based on the predicted genre
    fetch_song_suggestions(genre)

def fetch_song_suggestions(genre):
    # Example of a simple song suggestion API
    # Replace with an actual API URL that returns song suggestions based on genre
    url = f"http://example.com/api/songs?genre={genre}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            suggestions = response.json()  # Assuming the API returns a JSON response
            suggestion_text = "\n".join([f"{song['title']} by {song['artist']}" for song in suggestions])
            messagebox.showinfo("Song Suggestions", f"Suggested Songs:\n{suggestion_text}")
        else:
            messagebox.showerror("Error", "Could not fetch song suggestions.")
    except requests.RequestException as e:
        messagebox.showerror("Error", f"Request failed: {str(e)}")

# Save recording
def save_audio():
    global last_audio
    if last_audio is not None:
        file_name = f"recording_{random.randint(1000, 9999)}.wav"
        sf.write(file_name, last_audio, SAMPLE_RATE)
        audio = AudioSegment.from_wav(file_name)
        mp3_filename = file_name.replace('.wav', '.mp3')
        audio.export(mp3_filename, format="mp3", bitrate="320k")
        
        messagebox.showinfo("Save Recording", f"Audio saved as {mp3_filename} at 320 kbps")
        status_label.config(text="Audio Saved at 320 kbps")
    else:
        messagebox.showerror("Error", "No audio recorded to save.")

# Play recorded audio
def play_audio():
    global last_audio
    if last_audio is not None:
        status_label.config(text="Playing audio...")
        sd.play(last_audio, samplerate=SAMPLE_RATE)
        sd.wait()
        status_label.config(text="Playback finished.")
    else:
        messagebox.showerror("Error", "No audio recorded to play.")

# Update classification history table
def update_history_table():
    for row in history_tree.get_children():
        history_tree.delete(row)
    for record in classification_history:
        history_tree.insert("", "end", values=record)

# Generate confusion matrix
def plot_confusion_matrix():
    if not classification_history:
        messagebox.showinfo("Info", "No classifications to generate confusion matrix.")
        return
    
    genres = [rec[1] for rec in classification_history]
    unique_genres = sorted(set(genres))
    cm = confusion_matrix(genres, genres, labels=unique_genres)
    
    fig, ax = plt.subplots(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=unique_genres, yticklabels=unique_genres, cmap="Blues")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    canvas = FigureCanvasTkAgg(fig, master=tab_results)
    canvas.get_tk_widget().pack(fill="both", expand=True)
    canvas.draw()

# Display spectrogram
def plot_spectrogram():
    if last_audio is not None:
        fig, ax = plt.subplots(figsize=(6, 5))
        S = librosa.feature.melspectrogram(y=last_audio, sr=SAMPLE_RATE, n_fft=N_FFT, hop_length=HOP_LENGTH)
        S_dB = librosa.power_to_db(S, ref=np.max)
        img = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=SAMPLE_RATE, fmax=8000, ax=ax)
        fig.colorbar(img, ax=ax, format='%+2.0f dB')
        ax.set_title("Mel Spectrogram")
        canvas = FigureCanvasTkAgg(fig, master=tab_visualization)
        canvas.get_tk_widget().pack(fill="both", expand=True)
        canvas.draw()

# Display MFCC
def plot_mfcc():
    if last_audio is not None:
        mfcc = librosa.feature.mfcc(y=last_audio, sr=SAMPLE_RATE, n_mfcc=NUM_MFCC)
        fig, ax = plt.subplots(figsize=(6, 5))
        img = librosa.display.specshow(mfcc, x_axis='time', sr=SAMPLE_RATE, ax=ax)
        ax.set_title("MFCC")
        fig.colorbar(img, ax=ax)
        canvas = FigureCanvasTkAgg(fig, master=tab_visualization)
        canvas.get_tk_widget().pack(fill="both", expand=True)
        canvas.draw()

# Exit application
def exit_application():
    root.quit()

# GUI Layout for Record & Classify
frame = ttk.Frame(tab_main)
frame.pack(pady=10)

result_label = ttk.Label(frame, text="Predicted Genre will appear here", font=("Helvetica", 12), background="#1e1e2e", foreground="#00ff00")
result_label.pack(pady=20)

status_label = ttk.Label(frame, text="Status: Waiting...", font=("Helvetica", 10), background="#1e1e2e", foreground="#ffdd00")
status_label.pack(pady=10)

btn_frame = ttk.Frame(frame)
btn_frame.pack(pady=10)

ttk.Button(btn_frame, text="Record and Classify", command=classify_audio).grid(row=0, column=0, padx=5)
ttk.Button(btn_frame, text="Save Recording", command=save_audio).grid(row=0, column=1, padx=5)
ttk.Button(btn_frame, text="Play Recording", command=play_audio).grid(row=0, column=2, padx=5)

# Visualization Tab Layout
visualization_frame = ttk.Frame(tab_visualization)
visualization_frame.pack(pady=10)

ttk.Button(visualization_frame, text="Plot Spectrogram", command=plot_spectrogram).pack(side="left", padx=5)
ttk.Button(visualization_frame, text="Plot MFCC", command=plot_mfcc).pack(side="left", padx=5)

# Results Tab Layout
history_tree = ttk.Treeview(tab_results, columns=("Timestamp", "Genre", "Confidence"), show="headings")
history_tree.heading("Timestamp", text="Timestamp")
history_tree.heading("Genre", text="Genre")
history_tree.heading("Confidence", text="Confidence (%)")
history_tree.pack(expand=True, fill='both', padx=10, pady=10)

# Buttons for results tab
ttk.Button(tab_results, text="Generate Confusion Matrix", command=plot_confusion_matrix).pack(pady=10)
ttk.Button(tab_results, text="Exit", command=exit_application).pack(pady=10)

# Run the GUI loop
root.mainloop()
